IMPORTAMOS LAS LIBRERIAS NECESARIAS PARA NORMALIZAR EL DATASET DE "USER_ITEM"

In [9]:
import pandas as pd
import ast  

SE ABRE EL ARCHIVO .JSON DE LA SIGUIENTE MANERA, PRIMERO SE DESANIDAN LOS DATOS Y LUEGO SE CONVIERTE EN UN DATAFRAME

In [10]:
ruta=r'C:\Users\User\OneDrive\Documents\Trabajo\Henry\Proyecto Ind. 1\Datos Crudos\australian_users_items.json'
data=[]

with open(ruta, 'r', encoding='utf-8') as file:
    for linea in file:
        try:
            json_datos=ast.literal_eval(linea)
            data.append(json_datos)
        except ValueError:
            print(f"Error en la linea: {linea}")
            continue

df_items=pd.DataFrame(data)

In [11]:
df_items.drop(columns=['steam_id','user_url'], inplace=True)

NORMALIZAMOS EL DATASET

In [12]:
df_explode = df_items.explode('items').reset_index(drop=True)
items = pd.json_normalize(df_explode['items']).reset_index(drop=True)
df_items_full = pd.concat([df_explode.drop('items', axis=1), items], axis=1)

ELIMINAMOS NUEVAMENTE LA NUEVAS COLUMNAS INNECESARIAS GENERADAS DESPUES DE LA NORMALIZACION

In [13]:
df_items_full.drop(columns=['item_id','item_name'], inplace=True)

ELIMINAMOS LOS VALORES DUPLICADOS MANTENIENDO EL PRIMER VALOR DE LA COINCIDENCIA

In [14]:
df_items_clear=df_items_full.drop_duplicates(keep='first')

GUARDAMOS EL DATAFRAME COMO UN ARCHIVO DE .PARQUET

In [15]:
df_items_clear.head()

,user_id,items_count,playtime_forever,playtime_2weeks
0,76561197970982479,277,6.0,0.0
1,76561197970982479,277,0.0,0.0
2,76561197970982479,277,7.0,0.0
8,76561197970982479,277,4733.0,0.0
9,76561197970982479,277,1853.0,0.0


In [16]:
df_items_clear.to_parquet(r'C:\Users\User\OneDrive\Documents\Trabajo\Henry\Proyecto Ind. 1\Datos Limpios\user_items.parquet',compression='gzip')